##### <div align="right"> $07$ de Septiembre del $2021$ </div>

<img src=/images/ufro_logo.png width="400">

____

<center>
    
 ## 'Proyecto EFD MS-823: Evaluación de políticas ambientales. Impactos del programa de recambio de calefactores en hogares urbanos del centro-sur de Chile'.
    
<center>

____

# Resumen 

El Proyecto el cual ha sido objeto de análisis corresponde, de forma preliminar, a un estudio en el sector Urbano de Temuco acerca de la utilización y comparación de sistemas de calefacción específicos, en este caso estufas de combustión lenta frente a las estufas a pellet. Para lograr llevar a cabo tal comparativa, es requerido un equipo que proporcione parámetros específicos y automatizados de medición, en nuestro caso, los instrumentos utilizados durante el proyecto consistían en sensores a base de un **microcontrolador Arduino**, el cual contenía componentes de medición para obtener la hora de dicha medición, Temperatura, Humedad y Material Particulado(PM) a dos diferentes niveles de masa(fino y grueso), éste sensor se instalaba dentro de la casa de estudio en una zona no muy cercana a la calefacción, el cual se le asigna una etiqueta `IN` y otro sensor fuera de ésta, el cual tiene una etiqueta `OUT`, para tener un parámetro de referencia en la intemperie. Además, un chip sensor de altas temperaturas el que se denominará **SUM**, el cual se instalaba sobre la superficie del sistema de calefacción para así poder estudiar el comportamiento de la estufa, el cual mide la Temperatura y la hora de la medición.
   
Se disponían de alrededor de $90$ sensores Arduino y una cantidad considerable de sensores SUM para realizar las mediciones en un total de $396$ hogares. Para cumplir este objetivo, lo primero fue recolectar la información, proceso que duró $3$ semanas aproximadamente, durante ese tiempo, una empresa externa instalaba los sensores como se mencionó anteriormente, éstos realizaban lecturas durante $48$ horas y al momento de retornar del domicilio se desinfectaban y se les extraía la información en formato `.csv` en una carpeta con el código del hogar, donde si el archivo comienza con una letra `P` significa que es calefacción a Pellet y una `L` si correspondía a una casa a Leña, dicha letra seguida de un código único de identificación. Finalmente se eliminaba la información del sensor para hacer un chequeo general y que puedan seguir realizando mediciones en una casa diferente.

Durante este proceso de extracción de información, se comienza a generar e implementar un código Python para pre-procesar la data que se encontraba en bruto de una forma poco manejable. Finalmente, se logra reducir la data de $2$ GB aproximadamente de infomación a $60$ MB.



## Indice
_____

1. [Librerías](#id1)
2. [Procesamiento de datos sensor SUM](#id2)
3. [Procesamiento de datos de sensores Arduino](#id3)
4. [Hora de entrada real al domicilio](#id4)
5. [Automatización del procesamiento de datos](#id5)


<div id='id1' />


## Librerías
_____

Las librerías utilizadas en el desarrollo de la práctica se muestran a continuación:



In [1]:
#Librería de manejo de datos
import pandas as pd

#librería de algebra lineal y métodos matemáticos
import numpy as np

#Librerías de manejo de tiempo
import datetime
from time import time

import csv

#Lector y editor de directorios
import os 
import shutil

<div id='id2' />

## Procesamiento de datos sensor SUM

____

En primera instancia, nos encontramos con los sensores SUM, donde su formato se encuentra en series de tiempo y además la columna de registro de temperatura, en algunos sensores los datos tienen un decimal que el lector de `pandas` lo distingue como un dato extra (Debido al formato csv), que realmente no se debería considerar. Esto se puede ver en la siguiente línea de código para el sensor SUM Folio `P6`
____

In [2]:
try:
    pd.read_csv(r'D:/Mediciones/P6/DATAP6sum.csv')
except pd.errors.ParserError:
    print(pd.errors.ParserError)

<class 'pandas.errors.ParserError'>


Es por esto que se crea una función para abrir de forma que `pandas` no tenga error con los separadores con cada uno de estos archivos.

In [3]:
def openSUM(ruta):
    with open (ruta) as SUM:
        s=csv.reader(SUM)
        DF=[]
                            
        for columna in s:
            if str(columna).count(',')>2:
                columna=columna[:-1]
                DF.append(columna)
            else:
                DF.append(columna)
    
        df=(pd.DataFrame(DF[20:],columns=DF[19]))
        return df

In [4]:
df_sum0=openSUM(r'D:/Mediciones/L140839/DATAL140839sum.csv')

In [5]:
df_sum0

,Date/Time,Unit,Value
0,30-06-21 12:50:01,C,20
1,30-06-21 12:53:01,C,23
2,30-06-21 12:56:01,C,24
3,30-06-21 12:59:01,C,25
4,30-06-21 13:02:01,C,24
...,...,...,...
1405,03-07-21 11:05:01,C,12
1406,03-07-21 11:08:01,C,14
1407,03-07-21 11:11:01,C,16
1408,03-07-21 11:14:01,C,16


In [6]:
def PreprocesingSUM(df):
    FE=[]
    PData=df.iloc[:,lambda df:[0, 2]]
    
    for i in range(0,len(PData['Date/Time'])):
        
        Fecha=pd.to_datetime(PData['Date/Time'][i])
        FE.append(Fecha)
        
    PData=PData.drop(['Date/Time'],axis=1)
    PData['Fecha y Hora']=FE
    
    return PData

In [7]:
PreprocesingSUM(df_sum0)

,Value,Fecha y Hora
0,20,2021-06-30 12:50:01
1,23,2021-06-30 12:53:01
2,24,2021-06-30 12:56:01
3,25,2021-06-30 12:59:01
4,24,2021-06-30 13:02:01
...,...,...
1405,12,2021-03-07 11:05:01
1406,14,2021-03-07 11:08:01
1407,16,2021-03-07 11:11:01
1408,16,2021-03-07 11:14:01


Se logra apreciar que aún no reducimos la data, sólo la hicimos más manejable.



<div id='id3' />

## Procesamiento de datos de sensores Arduino
_____


In [8]:
df0=pd.read_csv(r'D:\Mediciones\L140839\DATAL140839IN.csv',header=None)
print(df0)

                    0  1                   2      3      4   5   6
0     sensor-Cristian  1  2021-6-30 12:53:23  64.70  17.50  11  34
1     sensor-Cristian  2  2021-6-30 12:53:26  63.30  18.40  17  25
2     sensor-Cristian  3  2021-6-30 12:53:29  63.10  18.40  17  25
3     sensor-Cristian  1  2021-6-30 12:56:28  63.00  18.40  18  26
4     sensor-Cristian  2  2021-6-30 12:56:31  61.60  18.90  17  27
...               ... ..                 ...    ...    ...  ..  ..
3372  sensor-Cristian  2    2021-7-3 3:28:16  67.70  13.60   2   2
3373  sensor-Cristian  3    2021-7-3 3:28:19  67.20  13.60   2   3
3374  sensor-Cristian  1    2021-7-3 3:31:18  67.70  13.60   2   3
3375  sensor-Cristian  2    2021-7-3 3:31:21  67.70  13.60   2   3
3376  sensor-Cristian  3    2021-7-3 3:31:24  67.70  13.60   2   2

[3377 rows x 7 columns]


 
Aquí, cada índice en las culmnas tienen asignado un parámetro de medición:
 
   1.  Frecuencia de medición en un intervalo de tiempo, entre $[ 1,3]$.

   2.  Fecha y hora de la medición.

   3.  Humedad relativa $[\% ] $.

   4.  Temperatura$[C°]$.

   5.  Material particulado a $2.5 [ \frac{\mu g}{L}]$.

   6.  Material particulado a $10 [\frac{\mu g}{L}]$.

Para comenzar, se van a promediar las frecuencias de medición para disminuir la data, el tamaño luego de procesar será a lo más $\frac{1}{3}$ del tamaño de la data original.

In [9]:
def PreprocesingArduino(df):
    
    #creamos la data con valores nulos
    df.replace(' NAN',np.NaN,inplace=True)

    #Eliminamos los valores nulos de la columna de Temperatura, que es la que más se debe resguardar.
    df=df[df[4].notna()]
    
    F=[]
    H=[]
    HP=[]
    TP=[]
    PM2_5=[]
    PM10=[]
    j=0
    
    for i in range(0,len(df)):
        
        try:      
            #Tomamos el mayor valor de medición de la frecuencia(como máximo 3)
            
            a=float(int(max(df[1][j],df[1][j+1],df[1][j+2]))) 
            if df[1][j+1]==df[1][j]:
                 a=1
            
            if a==1:
                HP.append(float(df[3][j]))
                TP.append(float(df[4][j]))
                PM2_5.append(float(df[5][j]))
                PM10.append(float(df[6][j]))
                
            if a==2:
                HP.append(np.mean([float(df[3][j]),float(df[3][j+1])])) 
                TP.append(np.mean([float(df[4][j]),float(df[4][j+1])]))
                PM2_5.append(np.mean([float(df[5][j]),float(df[5][j+1])]))
                PM10.append(np.mean([float(df[6][j]),float(df[6][j+1])]))
                
                
            if a==3:
                HP.append(np.mean([float(df[3][j]),float(df[3][j+1]),float(df[3][j+2])]))
                TP.append(np.mean([float(df[4][j]),float(df[4][j+1]),float(df[4][j+2])]))
                PM2_5.append(np.mean([float(df[5][j]),float(df[5][j+1]),float(df[5][j+2])]))
                PM10.append(np.mean([float(df[6][j]),float(df[6][j+1]),float(df[6][j+2])]))
                 
            #De la columna tiempo, se extrae la hora  
            
            FH=df[2][j-1+a].split()
            f=FH[0].split('-')
            h=FH[1].split(':')
            
            if int(f[0])>2021 or int(f[1])>12 or int(h[0])>23 or int(h[1])>59:
                
                fh=F[-1]+datetime.timedelta(minutes=1)
                h=datetime.time(int(str(H[-1]).split(':')[0]),int(str(H[-1]).split(':')[1]),int(str(H[-1]).split(':')[2]))
                
            else:
                fh=datetime.datetime(int(f[0]),int(f[1]),int(f[2]),int(h[0]),int(h[1]),int(h[2]))
                h=FH[1]
            
            F.append(fh)
            H.append(h)
            
            if j<len(df)-a:    
                    j=j+a    
    
        except KeyError:
            i+=1
            j+=1
            
    index=0
    for i in range(0,len(F)-1):
        if F[i]==F[i+1]:
            index=i+1

            break
        else:
            index=len(FH)
            
     
    #Creamos el nuevo Data Frame con una cantidad considerablemente menor de datos
    
    PData=pd.DataFrame({'Fecha y Hora':F[0:index],
                        'H p':HP[0:index],
                        'T° p':TP[0:index],
                        'PM 2,5':PM2_5[0:index],
                        'PM 10':PM10[0:index]})
    
    return PData

Con esta función, podemos procesar nuestros datos de la línea  de código `In[8]`

In [10]:
PreprocesingArduino(df0)

,Fecha y Hora,H p,T° p,"PM 2,5",PM 10
0,2021-06-30 12:53:29,63.700000,18.100000,15.000000,28.000000
1,2021-06-30 12:56:35,62.066667,18.733333,17.666667,26.666667
2,2021-06-30 12:59:40,60.333333,19.433333,17.000000,25.000000
3,2021-06-30 13:02:45,60.433333,19.400000,17.666667,35.666667
4,2021-06-30 13:05:50,61.600000,19.033333,11.666667,22.666667
...,...,...,...,...,...
1184,2021-07-03 02:57:25,68.100000,13.700000,2.666667,4.333333
1185,2021-07-03 03:00:30,68.000000,13.666667,2.000000,5.333333
1186,2021-07-03 03:03:35,68.000000,13.666667,2.000000,5.333333
1187,2021-07-03 03:09:39,67.966667,13.633333,2.000000,3.666667


Esta data, por ejemplo, se redujo de 3377 mediciones, a sólo 1189. Una reducción del 65%

Haciendo una pequeña comparativa, el tamaño de los datos procesados del sensor Arduino es bastante similar al tamaño del sensor SUM, además de las horas de activación ser coincidentes, es claro que el sensor SUM estuvo por más tiempo guardando mediciones que es debido netamente a la durabilidad de la fuente de energía de cada sensor.

<div id='id4' />

## Hora de entrada real al domicilio
____

Es cierto que se requiere cierta rigurosidad a la hora de tomar como valor de entrada una fecha, este caso no es una excepción: Como ya se ha visto, la data captada por los sensores contienen fechas de medición, lo cual nos simplifica el hecho de encontrar el dato que se requiere como inicio, en nuestro caso, es la hora de entrada del sensor al domicilio. Por fortuna, tenemos acceso a la data que contiene las fechas y horas de entrada de cada domiclio gracias al código único.

In [11]:
Entradas=pd.read_excel(r'D:/Fechas_Temuco_IngresoSalidas.xlsx')

In [12]:
Entradas

,ID_FOLIO,TIPO,ARTEFACTO,CIUDAD,FECHA Instalacion,HORARIO Instalacion,FECHA retiro,HORARIO retiro
0,P-1130,PELLET - NO AISLADO,PELLET,Temuco,2021-07-08,16:10:00,2021-07-10,16:30:00
1,P-1247,PELLET - NO AISLADO,PELLET,Temuco,2021-07-07,14:20:00,2021-07-09,16:00:00
2,P-1613,PELLET - NO AISLADO,PELLET,Temuco,2021-06-28,12:50:00,2021-06-30,12:50:00
3,P-1537,PELLET - NO AISLADO,PELLET,Temuco,2021-07-03,15:15:00,2021-07-05,15:40:00
4,P-1773,PELLET - NO AISLADO,PELLET,Temuco,2021-07-02,14:27:00,2021-07-04,16:55:00
...,...,...,...,...,...,...,...,...
391,139848,LEÑA - AISLADO,LEÑA,TEMUCO,2021-07-26,12:30:00,2021-07-28,12:40:00
392,137008,LEÑA - AISLADO,LEÑA,TEMUCO,2021-07-26,10:35:00,2021-07-28,12:35:00
393,142536,LEÑA - AISLADO,LEÑA,TEMUCO,2021-07-26,14:50:00,2021-07-28,18:20:00
394,141127,LEÑA - AISLADO,LEÑA,TEMUCO,2021-07-23,09:50:00,2021-07-25,21:15:00


Este dataframe contiene información extra que para efectos de este informe, no son abordados. Es por esto que debemos ser selectivos con la información que necesitamos, que es el código de la casa y la hora de instalación del equimamiento de medición. 

In [13]:
Fecha_entrada=Entradas[['ID_FOLIO','FECHA Instalacion','HORARIO Instalacion']]

Aquí, podemos ver en la primera columna el Folio de cada una de las $396$ casas y su fecha y hora de instalación. Nuestro objetivo ahora es generar datos utilizables en pandas. 

Además, se ve por formato de los datos que el `ID_FOLIO` para las casas a leña no se encuentra en el formato debido, por lo que debemos modificar aquellos folios.

In [14]:
FOLIO=[]
Fechas_entrada=[]

for i in range(0,len(Entradas)):
    folio=str(Entradas['ID_FOLIO'][i])
    
    if folio[0]=='P':
        stnd_folio=(folio.split('-')[0]+folio.split('-')[1]).upper()
    else:
        stnd_folio='L'+folio
    
    f=str(Entradas['FECHA Instalacion'][i]).split()[0]
    h=str(Entradas['HORARIO Instalacion'][i])
    fyh=pd.to_datetime(f+' '+h)
    Fechas_entrada.append(fyh)
    FOLIO.append(stnd_folio)

df_fecha_entrada=pd.DataFrame({'Folio':FOLIO,
                               'Fecha y Hora entrada':Fechas_entrada
                               })

In [15]:
df_fecha_entrada

,Folio,Fecha y Hora entrada
0,P1130,2021-07-08 16:10:00
1,P1247,2021-07-07 14:20:00
2,P1613,2021-06-28 12:50:00
3,P1537,2021-07-03 15:15:00
4,P1773,2021-07-02 14:27:00
...,...,...
391,L139848,2021-07-26 12:30:00
392,L137008,2021-07-26 10:35:00
393,L142536,2021-07-26 14:50:00
394,L141127,2021-07-23 09:50:00


Así obtenemos un formato pulcro con objetos de tiempo operables gracias a la librería `datetime`

Ahora que tenemos la fecha y hora de entrada de los sensores a los domicilios, podemos buscar coincidencias en la data que generaron los sensores. Para conseguir esto, no basta con tomar la hora y utilizar `.loc` de pandas, ya que `.loc` solamente encuentra coincidencias exactas, lo que no podemos asegurar en el tiempo, es por esto que debemos partir de lo más general: el mes, por ejemplo. 

In [16]:
def obtener_hora(dataframe,fecha_hora_entrada):
    
    for t in range(0,len(dataframe)):
        
        dfa=pd.to_datetime(dataframe['Fecha y Hora'][t])-datetime.timedelta(seconds=pd.to_datetime(dataframe['Fecha y Hora'][t]).second)
        dfa=dfa.to_pydatetime()
        
        #buscamos coincidencias con la fecha de entrada en el dataframe
        
        if dfa.year==fecha_hora_entrada.year and dfa.month==fecha_hora_entrada.month and dfa.day==fecha_hora_entrada.day:
            
            if dfa.hour==fecha_hora_entrada.hour:
                
                if dfa.minute==fecha_hora_entrada.minute:
                    
                    return dataframe['Fecha y Hora'][t]
                    break
                
                else:
                    
                    dif=abs(dfa.minute-fecha_hora_entrada.minute)
                    
                    if dif<3:
                        return dataframe['Fecha y Hora'][t]
                        break
                    else:
                        return pd.to_datetime(dataframe['Fecha y Hora'][t])+datetime.timedelta(minutes=dif)
                        break

Así, con una fecha cualquiera, podemos encontrar coincidencias en la columna `Fecha y Hora` de los sensores procesados Arduino y Sum.

In [17]:
#Hora de prueba(dentro del rango del dataframe df0)=2021-06-30 12:58:01

fecha_de_prueba=datetime.datetime(2021,6,30,12,58,1)
obtener_hora(PreprocesingArduino(df0),fecha_de_prueba)


Timestamp('2021-06-30 12:58:29')

También es cierto que la entrada a la casa es una hora estimada de ingreso y no va a ser del todo exacta, es por esto que se crea un método que comenzará de un $\Delta t$ antes de la hora de entrada en la data.

Un parámetro útil para comprobar el ingreso al domicilio es el material particulado `PM 10`, ya que los sensores al ingresar al hogar se deben sacar de una bolsa plástica que los contiene a todos ellos(Sensor Arduino `IN`,`OUT` y sensor `SUM`) la cual no realiza medición de material particulado porque la bolsa se encuentra cerrada, sólamente al salir de la bolsa realiza la medición real de la calidad del aire, y es ahí que se debería generar una diferencia considerable entre la medición actual y una fracción de tiempo antes(justo antes de salir de la bolsa). Para realizar todo esto, creamos un nuevo método para extraer la hora real de comienzo de medición en la casa y obtener una medición de $48$ horas.

In [18]:
def extraccion(dataframe,fecha_hora_entrada):
    
    fecha_hora_entrada=fecha_hora_entrada-datetime.timedelta(minutes=15)
    
    dataframe=dataframe.infer_objects()
    
    entrada=obtener_hora(dataframe,fecha_hora_entrada)
    
    df=dataframe.set_index('Fecha y Hora')
                    
    DF=df.loc[entrada:]
    
    indice=len(dataframe)-len(DF)
   
    count=0
    
    for i in range(1,len(dataframe)+1):
        
        dif=abs(DF['PM 10'][i]-DF['PM 10'][count])

        if dif>=20 and count<40:
            start=dataframe.iloc[i+indice]['Fecha y Hora']
            break
         
        if dif>7 and count>40  :
            start=dataframe.iloc[i+indice]['Fecha y Hora']
            break
    
        if count>40  :
            start=fecha_hora_entrada
            break
        
        count+=1
    end=datetime.datetime.strptime(str(start),"%Y-%m-%d %H:%M:%S")+datetime.timedelta(hours=48)    
    Final=DF.loc[start:end]
    
    return Final

In [19]:
df_fecha_entrada.set_index('Folio').loc['L140839']

Fecha y Hora entrada   2021-06-30 15:07:00
Name: L140839, dtype: datetime64[ns]

In [20]:
fecha_entrada_df0=datetime.datetime(2021,6,30,15,8,0)

fecha_real=obtener_hora(PreprocesingArduino(df0),fecha_entrada_df0)
extraccion(PreprocesingArduino(df0),fecha_real)

,H p,T° p,"PM 2,5",PM 10
Fecha y Hora,,,,
2021-06-30 15:09:44,64.066667,20.400000,41.333333,63.666667
2021-06-30 15:12:46,63.250000,20.800000,67.000000,115.000000
2021-06-30 15:15:55,60.866667,20.866667,73.666667,119.000000
2021-06-30 15:19:00,61.133333,20.766667,75.666667,135.666667
2021-06-30 15:22:05,61.366667,20.733333,71.666667,123.000000
...,...,...,...,...
2021-07-02 14:55:17,57.800000,20.300000,19.000000,40.500000
2021-07-02 14:58:26,57.833333,20.333333,18.333333,34.333333
2021-07-02 15:01:29,57.800000,20.300000,18.500000,33.500000


Con todas herramientas creadas, es posible reducir una data originalmente de $3377$ datos a una de sólamente $917$, todos utilizables.

Del mismo modo, de una forma no tan elegante podemos extraer los datos utilizables del sensor SUM.

In [21]:
Fecha_in=fecha_entrada_df0
Fecha_out=fecha_entrada_df0+datetime.timedelta(hours=48)

df_sump=PreprocesingSUM(df_sum0)

hora_sum_in=pd.to_datetime(obtener_hora(df_sump,Fecha_in)) 

df_sum_limpio=df_sump.set_index('Fecha y Hora')


In [22]:
df_sum_limpio

,Value
Fecha y Hora,
2021-06-30 12:50:01,20
2021-06-30 12:53:01,23
2021-06-30 12:56:01,24
2021-06-30 12:59:01,25
2021-06-30 13:02:01,24
...,...
2021-03-07 11:05:01,12
2021-03-07 11:08:01,14
2021-03-07 11:11:01,16


Aquí, podemos notar un error en el registro de la fecha y hora de nuestra data del sensor SUM, se puede apreciar que los datos han sido mal escritos, específicamente el mes y el día han sido permutados en algún momento. Se debe realizar un trato especial con este tipo de archivos.

In [23]:
F=[]

for fecha in df_sump['Fecha y Hora']:
    if str(fecha.date()).split('-')[1]!='06':
        #if str(fecha.date()).split('-')[2]=='01'
        fecha_correcta=datetime.datetime.strptime(str(fecha.date()).split('-')[0]+'-'+str(fecha.date()).split('-')[2]+'-'+str(fecha.date()).split('-')[1]+' '+str(fecha.time()),"%Y-%m-%d %H:%M:%S")
        F.append(fecha_correcta)
    else:
        F.append(fecha)

df_sump['Fecha y Hora']=F


In [24]:
df_sump.iloc[45:]

,Value,Fecha y Hora
45,19,2021-06-30 15:05:01
46,19,2021-06-30 15:08:01
47,20,2021-06-30 15:11:01
48,26,2021-06-30 15:14:01
49,26,2021-06-30 15:17:01
...,...,...
1405,12,2021-07-03 11:05:01
1406,14,2021-07-03 11:08:01
1407,16,2021-07-03 11:11:01
1408,16,2021-07-03 11:14:01


Ahora, podemos encontrar la hora de entrada y realizar 48 horas de medición

In [25]:
hora_sum_out=pd.to_datetime(obtener_hora(df_sump,Fecha_out))

df_sump.set_index('Fecha y Hora').loc[hora_sum_in:hora_sum_out]

,Value
Fecha y Hora,
2021-06-30 15:08:01,19
2021-06-30 15:11:01,20
2021-06-30 15:14:01,26
2021-06-30 15:17:01,26
2021-06-30 15:20:01,26
...,...
2021-07-02 14:56:01,23
2021-07-02 14:59:01,23
2021-07-02 15:02:01,23


<div id='id5' />

## Automatización del procesamiento de datos 
____

Gracias a todas las herramientas creadas a lo largo de este informe, es posible recorrer el directorio en donde tenemos almacenada nuestra data convenientemente agrupada de forma tal que sea posible acceder a una casa mediante el Folio y a su respectivo sensor `SUM`, `IN` u `OUT`.

Primero que todo, creamos la carpeta donde se van a almacenar las mediciones limpias utilizables

In [ ]:
try:
    shutil.rmtree(r'D:/Mediciones Limpias')
    os.mkdir(r'D:/Mediciones Limpias')
    
except FileNotFoundError:
    os.mkdir(r'D:/Mediciones Limpias')

Posteriormente, recorremos cada una de las 396 carpetas

In [ ]:
for i in range(0,len(df_fecha_entrada)):
    
    Fecha=df_fecha_entrada['Fecha y Hora entrada'][i].to_pydatetime()
    
    Folio=df_fecha_entrada['Folio'][i]
    
    ruta=r'D:/Mediciones/'+Folio
    
    Data=os.listdir(ruta)
    
    print(Folio)
    
    if Folio!='P97' and Folio!='P1773':
        
        for data in Data:
            
            if data.split('.')[1].upper()=='CSV':
                ruta_data=ruta+'/'+data
                
                
                if data.split('.')[0].upper().replace('DATA'+Folio,'')=='IN' or data.split('.')[0].upper().replace('DATA'+Folio,'')=='INT':
                    
                    if Folio!='L142764' and Folio!='L134130' and Folio!='L134801':
                        
                        if len(pd.read_csv(ruta_data,header=None))>100:
                            
                            df_in=PreprocesingArduino(pd.read_csv(ruta_data,header=None))
                            df_in_limpio=extraccion(df_in,Fecha)
                            df_in_limpio.to_csv(r'D:/Mediciones Limpias/'+Folio+'/'+Folio+'IN.csv',index='Fecha y Hora')
                            print('Archivo INT check')
                            
                        else:      
                            
                            txt_errin=open(r'D:/Mediciones Limpias/'+Folio+'/ERROR_'+Folio+'in.txt','w')
                            txt_errin.write('Data muy pequeña para procesar')
                            txt_errin.close()
            
                
                if data.split('.')[0].upper().replace('DATA'+Folio,'')=='OUT' or data.split('.')[0].upper().replace('DATA'+Folio,'')=='AUT':
                    
                    if Folio!='L140706' and Folio!='L140818' and Folio!='L140706' and Folio!='L141838' and Folio!='P40':
                        
                        if len(pd.read_csv(ruta_data,header=None))>100: 
                            
                            df_out=PreprocesingArduino(pd.read_csv(ruta_data,header=None))
                            df_out_limpio=extraccion(df_out,Fecha)
                            df_out_limpio.to_csv(r'D:/Mediciones Limpias/'+Folio+'/'+Folio+'OUT.csv',index='Fecha y Hora')
                            print('Archivo OUT check')
                        else:
                            
                            txt_errout=open(r'D:/Mediciones Limpias/'+Folio+'/ERROR_'+Folio+'out.txt','w')
                            txt_errout.write('Data muy pequeña para procesar')
                            txt_errout.close()
                            
                if data.split('.')[0].upper().replace('DATA'+Folio,'')=='SUM':
                    
                    if Folio!='P340':
                        
                        if len(openSUM(ruta_data))>100:
                            
                            df_sum=openSUM(ruta_data)
                            df_sump=PreprocesingSUM(df_sum)

                            hora_sum=obtener_hora(df_sump,Fecha)
                            hora_sum_out=obtener_hora(df_sump,Fecha+datetime.timedelta(hours=48))

                            df_sum_limpio=df_sump.set_index('Fecha y Hora').loc[hora_sum:hora_sum_out]
                            df_sum_limpio.to_csv(r'D:/Mediciones Limpias/'+Folio+'/'+Folio+'SUM.csv',index='Fecha y Hora')
                            
                            print('Archivo SUM '+Folio+' check')
                        else:
                            
                            txt_errin=open(r'D:/Mediciones Limpias/'+Folio+'/ERROR_'+Folio+'sum.txt','w')
                            txt_errin.write('Data muy pequeña para procesar')
                            txt_errin.close()
                            print('Archivo SUM '+Folio+' not check')
                            

Con este código, es posible limpiar cada uno de los datos de forma rápida y controlada, cuidando errores y encontrando singularidades, como lo son los archivos con Folio `P97` `P1773` `L142764` `L134130` `L134801` `L140706` `L140818` `L140706` `L141838` `P40` y `P340`. Los cuales presentan un error diferente al tamaño de los datos. Se le puede atribuir un error de lectura y/o de escritura del sensor. Estos errores se encuentran detallados en un archivo `Practica ll.py`.